In [68]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MeshCat as mc
import Distributions
import Random
import Combinatorics

using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using CSV
using DataFrames
using Distributions
using Combinatorics

  Activating environment at `c:\Users\AiPEX-WS4\Documents\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Project.toml`


### Sample parameters 

In [149]:
function generate_piano_params(N_param_samples)::Matrix{Float64}
    # Choose 3 notes to play from an octave range: 12_P_3
    notes = 1:12
    # notes_distribution = DiscreteUniform(1,12)
    notes_distribution = permutations(notes, 3)
    notes = rand(collect(notes_distribution), N_param_samples)
    notes = stack(notes)
    notes = transpose(notes)
    # @show size(notes)
    # @show typeof(notes)
    # @show notes
    # println()


    # Choose note durations
    t_end = 5.0
    note_timings_distribution = Product([Uniform(0, t_end), Uniform(0, t_end)])
    notes_timings = rand(note_timings_distribution, N_param_samples)
    sort!(notes_timings,dims=1)
    notes_timings = transpose(notes_timings)
    # @show size(notes_timings)
    # @show typeof(notes_timings)
    # @show notes_timings
    # println()

    # Choose fingers to play 3 notes: 5_P_3
    finger_idx = 1:5
    # finger_distribution = DiscreteUniform(1,5)
    fingers_distribution = permutations(finger_idx, 3)
    fingers = rand(collect(fingers_distribution), N_param_samples)
    fingers = stack(fingers)
    fingers = transpose(fingers)
    # @show size(fingers)
    # @show typeof(fingers)
    # @show fingers

    # Joint parameter distribution sample
    θ_samples = hcat(notes, notes_timings, fingers)

    return θ_samples
end



generate_piano_params (generic function with 1 method)

In [160]:
df = DataFrame(params = Vector{Vector{Float64}}(), X=Vector{Vector{Vector{Float64}}}(), U=Vector{Vector{Vector{Float64}}}(), obj = Float64[], solve_time_sec = Float64[], term_status = MOI.TerminationStatusCode[])

Random.seed!(123)

N_param_samples = 20 # Number of parameters to sample for the OCP
nlp_params = generate_piano_params(N_param_samples)
# θ_samples = eachrow(θ_samples)
# @show θ_samples


for θ in eachrow(nlp_params)
    println(θ)
end

[12.0, 6.0, 5.0, 0.027347800755164187, 0.6836255505825772, 5.0, 1.0, 4.0]
[5.0, 4.0, 2.0, 0.7052942142950092, 4.738556660343784, 1.0, 4.0, 5.0]
[4.0, 3.0, 8.0, 3.6147614069933165, 3.917928500958822, 2.0, 3.0, 4.0]
[6.0, 10.0, 12.0, 0.07417653569472216, 3.9683817743069536, 3.0, 1.0, 2.0]
[8.0, 7.0, 4.0, 2.2342199914849523, 3.133995261499721, 4.0, 5.0, 1.0]
[7.0, 1.0, 10.0, 1.8458316830080712, 2.346509354352977, 5.0, 2.0, 4.0]
[6.0, 2.0, 1.0, 2.285682736570066, 2.5126155584440535, 1.0, 3.0, 2.0]
[11.0, 10.0, 5.0, 1.5655097295620113, 4.695850739023899, 1.0, 3.0, 5.0]
[4.0, 6.0, 1.0, 4.699540872998513, 4.98295211711779, 4.0, 3.0, 1.0]
[7.0, 6.0, 12.0, 0.7765297547886063, 2.45157885231851, 5.0, 1.0, 4.0]
[1.0, 6.0, 10.0, 1.0231561100611197, 2.1881605123463, 2.0, 3.0, 1.0]
[7.0, 8.0, 5.0, 1.0858587636848838, 2.675355736748318, 3.0, 5.0, 1.0]
[6.0, 5.0, 1.0, 0.3045117908439887, 0.5108241562351368, 4.0, 3.0, 5.0]
[2.0, 4.0, 7.0, 2.5816769701224263, 4.046913583847365, 3.0, 5.0, 4.0]
[8.0, 1.0, 

1×3 adjoint(::Vector{Int64}) with eltype Int64:
 1  2  3